In [8]:
import pandas as pd
import os
from tqdm import tqdm
DIR = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/'
labels_dir = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/yolov5l6_exp003/OOF/Fold0/Infer_2048_CONFTHRESH_45/labels'
submit = pd.read_csv(os.path.join(DIR,'SampleSubmission.csv'))
test_df = pd.read_csv(os.path.join(DIR,'Test.csv'))
train = pd.read_csv(os.path.join(DIR,'Train.csv'))
mapping= {0: 'abw',1: 'pbw'}

In [9]:
train.head(1
          )

,image_id_worm,worm_type,number_of_worms
0,id_0002ea6f15c7fa6f4c221783.jpg,pbw,51


In [19]:
from collections import defaultdict
train_files = defaultdict(list)

for folds in [0,1,2,3,4]:
    files = list(pd.read_csv(f'///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/splits/fold{folds}.txt',header=None)[0].apply(lambda x:x.split("/")[-1]))
    train_files[folds].append(files)

# train_files= ([item for sublist in train_files for item in sublist])
# fold= ([item for sublist in fold for item in sublist])
# fold_dict = dict(zip(train_files,fold))
# train['fold'] = train['image_id_worm'].map(fold_dict)

# train = train[~train['fold'].isna()].reset_index(drop=True)
# train['fold'] = train['fold'].astype(int)
# print(train['fold'].unique())

In [31]:
len(train_files[3][0])

1417

In [5]:
train.head()

,image_id_worm,worm_type,number_of_worms,fold
0,id_0002ea6f15c7fa6f4c221783.jpg,pbw,51,1
1,id_0005ef295aafe6acc63587db.jpg,pbw,8,0
2,id_00093f2c76f6488737325859.jpg,pbw,12,2
3,id_000b2e6c437c643f25d4a6c3.jpg,pbw,87,0
4,id_000c2040da4b05816cefbb96.jpg,NaN,0,0


In [33]:
train_files[3][0]

['id_bc86d3cfd0c0463fa5cae33e.jpg',
 'id_3a116e1b1f0bdd40abb1545c.jpg',
 'id_a998bd90f7d03ac1d668b552.jpg',
 'id_f11b846c5dc0d9b877850584.jpg',
 'id_bcf4e4a87efc312ecc890c7b.jpg',
 'id_a8718dd676c6c06f241857fc.jpg',
 'id_2e8cd44287db582e0d2d449d.jpg',
 'id_0a99918f6e85f734a00d969d.jpg',
 'id_ba9f1f75764f11d3b91744c9.jpg',
 'id_2a0ea4cf4ec97fc0da0853a8.jpg',
 'id_d4358cf715a205c77029e386.jpg',
 'id_43674145e558255dac36e2e9.jpg',
 'id_e11b0d905bbad80205cb06f6.jpg',
 'id_dd4a1471e045dd5636c28c49.jpg',
 'id_a6adee5dcccf0413df230a68.jpg',
 'id_a9e4984e6ea380f2ee9b1965.jpg',
 'id_65e314f0fb64bee004197730.jpg',
 'id_ec7f482c3e4a6a4d26d4cddc.jpg',
 'id_36dd8c9e500496d39ebff1c8.jpg',
 'id_c9fb2ac655d347d2a5edbe43.jpg',
 'id_7adfedff9dac11a3f90379a5.jpg',
 'id_64036cd3993164676b581fed.jpg',
 'id_52f3fd825a8a7ff9941bacb2.jpg',
 'id_4f8b845554c16eee06f32088.jpg',
 'id_6bbfc3c80f11f7e8bc7ebb6f.jpg',
 'id_a6ce0378755b32f4ced2fffb.jpg',
 'id_c5cbff44a04d3abc06ca312f.jpg',
 'id_67e6ad7107c09c27988c475

In [40]:
out = dict()
for i in range(5):
    labels_dir = f'///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/yolov5l6_exp003/OOF/Fold{i}/Infer_2048_CONFTHRESH_45/labels'
    files = train_files[i][0]
    for img_id in tqdm(files):
        img = img_id.split('.')[0]
        out[f'{img}_abw'] = 0
        out[f'{img}_pbw'] = 0

        # read text file into pandas DataFrame
        try:
            df = pd.read_csv(f"{labels_dir}/{img}.txt", sep=" ",header=None)
            df.rename(columns = {0:'labels'},inplace=True)
            out_temp = df['labels'].value_counts().to_dict()
            for k,v in out_temp.items():
                if k == 0:
                    out[f'{img}_abw'] += out_temp[k]
                if k == 1:
                    out[f'{img}_pbw'] += out_temp[k]        
        except:
            continue

100%|███████████████████████████████████████████████████████████████████████████████| 1417/1417 [00:20<00:00, 70.74it/s]


In [41]:
out_df = pd.DataFrame.from_dict(out,orient='index').reset_index()
out_df.rename(columns={'index':'image_id_worm',0:'number_of_worms'},inplace=True)
out_df

,image_id_worm,number_of_worms
0,id_8ada71185e18dc8838b4925b_abw,1
1,id_8ada71185e18dc8838b4925b_pbw,1
2,id_d625dfda0715dd288c485300_abw,0
3,id_d625dfda0715dd288c485300_pbw,4
4,id_dd9bf0420318654d9bc29a05_abw,1
...,...,...
14153,id_79b8996262110c02ad9cdd0e_pbw,67
14154,id_c5edc2c9e5350152ff072c5d_abw,0
14155,id_c5edc2c9e5350152ff072c5d_pbw,98
14156,id_a1dd3d1e2f405cd5914318f3_abw,0


In [42]:
out_df.tail(100)

,image_id_worm,number_of_worms
14058,id_ec81bb851987f6e9f00d6783_abw,0
14059,id_ec81bb851987f6e9f00d6783_pbw,2
14060,id_6f0328870dfd72a4d86d055e_abw,0
14061,id_6f0328870dfd72a4d86d055e_pbw,63
14062,id_4c740e933e7c2e58e2763e39_abw,0
...,...,...
14153,id_79b8996262110c02ad9cdd0e_pbw,67
14154,id_c5edc2c9e5350152ff072c5d_abw,0
14155,id_c5edc2c9e5350152ff072c5d_pbw,98
14156,id_a1dd3d1e2f405cd5914318f3_abw,0


In [45]:
out_df['image_id_worm'].apply(lambda x:x[-3:])

0        abw
1        pbw
2        abw
3        pbw
4        abw
        ... 
14153    pbw
14154    abw
14155    pbw
14156    abw
14157    pbw
Name: image_id_worm, Length: 14158, dtype: object

In [46]:
out_df['Type'] = out_df['image_id_worm'].apply(lambda x:x[-3:])
out_df['image_id_worm'] = out_df['image_id_worm'].apply(lambda x:x[:-4])

In [52]:
out_df.pivot(index='image_id_worm',columns='Type',values='number_of_worms').reset_index().to_csv('///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/yolov5l6_exp003/OOF/OOF.csv')

### Preds on test df

In [53]:
test_df.head()

,image_id_worm
0,id_00332970f80fa9a47a39516d.jpg
1,id_0035981bc3ae42eb5b57a317.jpg
2,id_005102f664b820f778291dee.jpg
3,id_0066456f5fb2cd858c69ab39.jpg
4,id_007159c1fa015ba6f394deeb.jpg


In [59]:
out = dict()
labels_dir = f'///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/yolov5l6_exp003/Ensemble/Infer_2048_CONFTHRESH_45_5_Folds/labels'
files = list(test_df['image_id_worm'])
for img_id in tqdm(files):
    img = img_id.split('.')[0]
    out[f'{img}_abw'] = 0
    out[f'{img}_pbw'] = 0

    # read text file into pandas DataFrame
    try:
        df = pd.read_csv(f"{labels_dir}/{img}.txt", sep=" ",header=None)
        df.rename(columns = {0:'labels'},inplace=True)
        out_temp = df['labels'].value_counts().to_dict()
        for k,v in out_temp.items():
            if k == 0:
                out[f'{img}_abw'] += out_temp[k]
            if k == 1:
                out[f'{img}_pbw'] += out_temp[k]        
    except:
        continue

100%|██████████████████████████████████████████████████████████████████████████████| 2803/2803 [00:27<00:00, 100.74it/s]


In [60]:
out_df = pd.DataFrame.from_dict(out,orient='index').reset_index()
out_df.rename(columns={'index':'image_id_worm',0:'number_of_worms'},inplace=True)
out_df

,image_id_worm,number_of_worms
0,id_00332970f80fa9a47a39516d_abw,4
1,id_00332970f80fa9a47a39516d_pbw,0
2,id_0035981bc3ae42eb5b57a317_abw,0
3,id_0035981bc3ae42eb5b57a317_pbw,21
4,id_005102f664b820f778291dee_abw,9
...,...,...
5601,id_ffbcb27fa549278f47505515_pbw,0
5602,id_ffc0e41e10b0c964d4a02811_abw,0
5603,id_ffc0e41e10b0c964d4a02811_pbw,0
5604,id_fff8c253115aacded09ad7ed_abw,0


In [61]:
out_df['Type'] = out_df['image_id_worm'].apply(lambda x:x[-3:])
out_df['image_id_worm'] = out_df['image_id_worm'].apply(lambda x:x[:-4])

In [62]:
out_df

,image_id_worm,number_of_worms,Type
0,id_00332970f80fa9a47a39516d,4,abw
1,id_00332970f80fa9a47a39516d,0,pbw
2,id_0035981bc3ae42eb5b57a317,0,abw
3,id_0035981bc3ae42eb5b57a317,21,pbw
4,id_005102f664b820f778291dee,9,abw
...,...,...,...
5601,id_ffbcb27fa549278f47505515,0,pbw
5602,id_ffc0e41e10b0c964d4a02811,0,abw
5603,id_ffc0e41e10b0c964d4a02811,0,pbw
5604,id_fff8c253115aacded09ad7ed,0,abw


In [64]:
out_df.pivot(index='image_id_worm',columns='Type',values='number_of_worms').reset_index().to_csv('///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/yolov5l6_exp003/Ensemble/Infer_2048_CONFTHRESH_45_5_Folds/test_preds.csv')

### Fin 